# Práctica 3. Optimización de un modelo de ingeniería usando un algoritmo genético

A menudo, en ingeniería tenemos simulaciones de un modelo y hay que optimizar sus parámetros sin disponer de una fórmula explícita que relacione los parámetros con el comportamiento del modelo. Nos plantearemos un problema de ese tipo: tenemos una función que calcula las propiedades de una hélice en función de una serie de parámetros, pero no conocemos los cálculos que hace internamente. Disponemos de una caja negra que es un modelo de simulación (créditos: Siro Moreno y Carlos Dorado)

Aunque no es imprescindible conocer todos los detalles del modelo sí es recomendable familiarizarnos con los parámetros de configuración de la hélice que vamos a optimizar. 

In [ ]:
from modelo.helice import *

In [ ]:
help(calcular_helice)

Lo primero que tendrás que hacer es definir una representación adecuada para los cromosomas que representan un diseño de la hélice en base a sus parámetros. Nuestra hélice depende de varios parámetros que queremos optimizar. Podemos decidir optimizar algunos mateniendo un valor controlado de otros. 

***Como sugerencia*** se proponen los siguientes parámetros de optimización: 

- omega (velocidad de rotación) (Entre 0 y 200 radianes/segundo)
- R (radio de la hélice) (Entre 0.1 y 2 metros)
- b (número de palas) (Entre 2 y 5 palas) 
- theta0 (ángulo de paso colectivo) (Entre -0.26 y 0.26 radianes)(*se corresponde a -15 y 15 grados*)
- p (parámetro de torsión) (Entre -5 y 20 grados)
- cuerda (anchura de la pala) (Entre 0.01 y 0.2 metros)

y fijar los parámetros:
- vz (velocidad de vuelo)
- h (altura de vuelo)

Ten en cuenta que necesitarás funciones de decodificación hay que transformar la representación binaria en variables con sentido físico. Por ejemplo, si el entero de la variable Omega está entre 0 y 1023 (10bits), pero la variable Omega real estará entre 0 y 200 radianes por segundo, el valor se calcula como:
    omega = genes[0] * 200 / 1023
    
del mismo modo, para R:
    R =  0.1 + genes[1] * 1.9 / 1023 #Obtendremos un radio entre 0.1 y 2 metros
    
El número de palas debe ser un entero:
    b = genes[2] + 2 #(entre 2 y 5 palas)
    
    
Para la función de fitness usaremos la función que calcula el desempeño del rotor según sus parámetros:

    T, P, efic, mach_punta = calcular_rotor(omega, vz, R, b, h...) #Introduce las variables que uses de parámetro.
                                                                # Consulta la ayuda para asegurarte de que usas el 
                                                                # formato correcto!
Recuerda que T es la tracción de la hélice, P es la Potencia consumida por la hélice, efic representa la eficiencia propulsiva de la hélice y mach_punta representa el mach en la punta de las palas. El número de mach en las puntas de la hélice se refiere a la velocidad relativa de las puntas de la hélice con respecto a la velocidad del sonido en el aire circundante.

Antes de seguir vamos a observar en las gráficas cómo cambian las características de la hélice (tracción, potencia, eficiencia y mach de las puntas) para cada valor de velocidad de vuelto (avance en m/s). Fijamos unos ciertos parámetros para ver el desempeño de la hélice.    

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

In [ ]:
vel = np.linspace(0, 30, 100)
# La función linspace() devuelve 100 valores espaciados uniformemente dentro del intervalo especificado [0, ] 
# zeros_like return an array of zeros with the same shape and type as a given array.
efic = np.zeros_like(vel)
T = np.zeros_like(vel)
P = np.zeros_like(vel)
mach = np.zeros_like(vel)
for i in range(len(vel)):
    T[i], P[i], efic[i], mach[i] = calcular_helice(130, vel[i], 0.5, 3)

In [ ]:
plt.plot(vel, efic)
plt.title('Eficiencia de la hélice')

In [ ]:
plt.plot(vel, T)
plt.title('Tracción de la hélice')

In [ ]:
plt.plot(vel, P)
plt.title('Potencia consumida')

In [ ]:
plt.plot(vel, mach)
plt.title('Mach en la punta de las palas')

Si quieres puedes observar también el impacto del resto de parámetros. 

Para la ejecución del algoritmo tendrás que definir una **función de fitness** adecuada según lo que decidas optimizar.

Por ejemplo, si buscáramos que tuviera la tracción máxima sin preocuparnos de nada más, el valor de fitnes sería simplemente igual al de T:

    fitness = T

Si queremos imponer restricciones, por ejemplo, que la potencia sea menor a 1000 watios, se pueden añadir sentencias del tipo:

    if P > 1000:
        fitness -= 1000
        
Si queremos optimizar varias salidas podemos hacerlo de manera ponderada:

    fitness = parámetro_importante * 10 + parámetro_poco_importante * 0.5
    
También se pueden combinar diferentes funciones no lineales:

    fitness = parámetro_1 * parámetro_2 - parámetro_3 **2 * log(parámetro_4)

Puedes elegir con qué objetivo quieres optimizar la hélice pero te hacemos algunas sugerencias de posibles objetivos de optimización:

    - Maximizar la tracción (manteniendo una tracción mínima de 30 Newtons) con el mínimo radio posible.
    - Mínima potencia consumida posible, máxima eficiencia propulsiva, y mínimo radio posible (sin ser este criterio muy importante), manteniendo una tracción mínima de 40 Newtons y un mach en la punta de las palas de como mucho 0.7
    - Máximizar la eficiencia cuando vuela a 70 m/s, y maximizar la tracción por encima de 50 Newtons para el despegue (vz = 0)    

### Se pide utilizar un algoritmo genético para encontrar la mejor configuración de parámetros para el modelo de hélice dado y comentar los resultados obtenidos. 

Documentar convenientemente el código y justificar las decisiones para la configuración del algoritmo genético. 